# Decision Tree Learning: College Admission

A decision tree a learned set of rules that allows us to make decisions on data.

We are going to look at the prosper loan dataset.  This dataset shows a history of loans made by Prosper.

In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark

## Step 1: Load the Data

In [ ]:
dataset = spark.read.csv("/data/college-admissions/admission-data.csv", 
                         header=True, inferSchema=True)
## clean 
dataset = dataset.na.drop()

dataset.printSchema()
dataset.show(20)

## Step 2 : Basic Analysis

### 2.1 Look at data split by 'admit' colunm
Looks like a resaonble distribution

In [ ]:
## TODO : group by 'admit' column
dataset.groupBy('???').count().show()

### 2.2 Summary of data
use 'describe' function

In [ ]:
dataset.describe().show()

## Step 3: Build feature vectors using VectorAssembler.

In [ ]:
from pyspark.ml.feature import VectorAssembler

## TODO : create a feature vector using the following attributes
## Hint :   'gpa' ,   'gre',   'rank'
assembler = VectorAssembler(inputCols=['???', '???', '???'], outputCol="features")
feature_vector = assembler.transform(dataset)

## TODO : set 'admit' as 'label' column
feature_vector = feature_vector.withColumn("label", feature_vector["???"])
feature_vector.show()

## Step 4: Split Data into training and test.

We will split our the data up into training and test.  (You know the drill by now).

**=> TODO: Split dataset into 80% training, 20% validation**


In [ ]:
## TODO :  Split the data into 80% training and 20% test sets 
## Hint :  0.8,   0.2
(training, test) =  feature_vector.randomSplit([???,   ???])
#(training, test) =  feature_vector.randomSplit([???,   ???], seed=123)  # seed can be any number
print("training set = " , training.count())
print("testing set = " , test.count())

## Step 5: Decision Tree

### 5.1 Create tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

## TODO : Create a DecisionTree model with maximum bins = 500
## 
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=???)


dt

### 5.2 Train Tree

In [ ]:
%%time

## TODO : train model on training dataset
## Hint : pass 'training' dataset name

print("training starting...")
model = dt.fit(???)
print("training done.")



### 5.3 Print Tree

In [ ]:

## print the model
print(model)
print()
print(model.toDebugString)

## Step 6 : Get Predictions on Test

In [ ]:
## TODO : Get predictions on test data
## Hint : 'test' set name
predictions_test = model.transform(???)
predictions_test.show()

## do a sample label
predictions_test.sampleBy(col="label", fractions={0: 0.5, 1: 0.5}, seed=123).show()

## Step 7: Evaluate the model.

Let us check to see how the model did, using accuracy as a measure.

In [ ]:
predictions_test = model.transform(???) # Hint : test
predictions_train = model.transform(???)  # Hint : training

### 7.1 Model Accuracy

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")

print("Training set accuracy = " , evaluator.evaluate(predictions_train))
print("Test set accuracy = " , evaluator.evaluate(predictions_test))

### 7.2 Confusion Matrix

In [ ]:
## Group by 'admit' 
cm = predictions_test.groupBy('admit').pivot('prediction', [0,1]).count().na.fill(0).orderBy('admit')
cm.show()

In [ ]:
import seaborn as sns

cm_pd = cm.toPandas()
cm_pd.set_index("admit", inplace=True)
# print(cm_pd)

# colormaps : cmap="YlGnBu" , cmap="Greens", cmap="Blues",  cmap="Reds"
sns.heatmap(cm_pd, annot=True, fmt=',', cmap="Blues").plot()

## Step 8: Do a Few Runs
- Click 'Cell --> Run All'
- Observe how the 'accuracy' output changes above
- Why?  Can we get the same accuracy all the times?

To get 'consistant' split for testing and random data you can set the seed variable

```python
(training, test) =  feature_vector.randomSplit([0.7,   0.3], seed=123)
```